In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import data_request as dr

In [4]:
df = dr.retrieve_data_request()
df

,output variable name,units,ag,long_name,standard_name,priority,comments,frequency
0,tas,K,i,Near-Surface Air Temperature,air_temperature,CORE,NaN,mon
1,tas,K,i,Near-Surface Air Temperature,air_temperature,CORE,NaN,day
2,tas,K,i,Near-Surface Air Temperature,air_temperature,CORE,NaN,1hr
3,tasmax,K,NaN,Daily Maximum Near-Surface Air Temperature,air_temperature,CORE,maximum from all integrated time steps per day,mon
4,tasmax,K,NaN,Daily Maximum Near-Surface Air Temperature,air_temperature,CORE,maximum from all integrated time steps per day,day
...,...,...,...,...,...,...,...,...
582,rootd,m,NaN,Maximum Root Depth,root_depth,TIER 2,NaN,fx
583,sftlaf,%,NaN,Percentage of the Grid Cell Occupied by Lake,lake_area_fraction,TIER 2,not in CMIP or in CF,fx
584,sfturf,%,NaN,Percentage of the Grid Cell Occupied by City,urban_area_fraction,TIER 2,not in CMIP or in CF,fx
585,dtb,m,NaN,Depth to Bedrock,bedrock_depth,TIER 2,not in CMIP or in CF (lower boundary of land s...,fx


In [5]:
df.groupby(["priority", "output variable name"])["frequency"].apply(list)

priority  output variable name
CORE      clt                     [mon, day, 1hr]
          evspsbl                 [mon, day, 1hr]
          hurs                    [mon, day, 1hr]
          huss                    [mon, day, 1hr]
          orog                               [fx]
                                       ...       
TIER 2    zg20                    [mon, day, 6hr]
          zg30                    [mon, day, 6hr]
          zg50                    [mon, day, 6hr]
          zg70                    [mon, day, 6hr]
          zg750                   [mon, day, 6hr]
Name: frequency, Length: 199, dtype: object

In [6]:
df.to_csv("cordex-cmip6-data-request.csv", index=False)

In [8]:
url = "https://raw.githubusercontent.com/PCMDI/cmip6-cmor-tables/master/Tables/CMIP6_3hr.json"

In [25]:
import json
from urllib.request import urlopen


def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)


def get_cmip6_cmor_table(table):
    url = f"https://raw.githubusercontent.com/PCMDI/cmip6-cmor-tables/master/Tables/CMIP6_{table}.json"
    return get_jsonparsed_data(url)


def get_variable_entry(variable, table):
    return get_cmip6_cmor_table(table)["variable_entry"][variable]


url = "https://raw.githubusercontent.com/PCMDI/cmip6-cmor-tables/master/Tables/CMIP6_3hr.json"
table_3hr = get_jsonparsed_data(url)

In [40]:
dr.table_map

{'mon': 'Amon', 'day': 'day', '6hr': '3hr', '3hr': '3hr', '1hr': '3hr'}

In [25]:
def update_cell_method(row):
    #
    table = dr.table_map[row["frequency"]]
    try:
        return dr.get_variable_entry(row["output variable name"], table)["cell_methods"]
    except Exception:
        return None


df["cell_methods"] = df.apply(update_cell_method, axis=1)

In [50]:
from pathlib import Path

Path("cmip6.json").suffix

'.json'

In [30]:
import pooch

pooch.make_registry("/Users/lars/python/cmip6-cmor-tables/Tables", "registry")

In [38]:
import github
from github import Github

g = Github()
repo = g.get_repo("PCMDI/cmip6-cmor-tables")
contents = repo.get_contents("Tables")
# repo = g.get_user().get_repo( "cmip6-cmor-tables" )
# repo.get_dir_contents("")

In [44]:
import os

os.path.basename(contents[0].path)

'CMIP6_3hr.json'

In [107]:
dr.get_cmip6_cmor_table("6hrLev")["variable_entry"]

{'bs550aer': {'frequency': '6hrPt',
  'modeling_realm': 'aerosol',
  'standard_name': 'volume_scattering_function_of_radiative_flux_in_air_due_to_ambient_aerosol_particles',
  'units': 'm-1 sr-1',
  'cell_methods': 'area: mean time: point',
  'cell_measures': 'area: areacella',
  'long_name': 'Aerosol Backscatter Coefficient',
  'comment': 'Aerosol  Backscatter at 550nm and 180 degrees, computed from extinction and lidar ratio',
  'dimensions': 'longitude latitude alevel time1 lambda550nm',
  'out_name': 'bs550aer',
  'type': 'real',
  'positive': '',
  'valid_min': '',
  'valid_max': '',
  'ok_min_mean_abs': '',
  'ok_max_mean_abs': ''},
 'ec550aer': {'frequency': '6hrPt',
  'modeling_realm': 'aerosol',
  'standard_name': 'volume_extinction_coefficient_in_air_due_to_ambient_aerosol_particles',
  'units': 'm-1',
  'cell_methods': 'area: mean time: point',
  'cell_measures': 'area: areacella',
  'long_name': 'Aerosol Extinction Coefficient',
  'comment': 'Aerosol volume extinction coeff

In [109]:
dr.cmip6_table_list()

variables


['CMIP6_3hr.json',
 'CMIP6_6hrLev.json',
 'CMIP6_6hrPlev.json',
 'CMIP6_6hrPlevPt.json',
 'CMIP6_AERday.json',
 'CMIP6_AERhr.json',
 'CMIP6_AERmon.json',
 'CMIP6_AERmonZ.json',
 'CMIP6_Amon.json',
 'CMIP6_CF3hr.json',
 'CMIP6_CFday.json',
 'CMIP6_CFmon.json',
 'CMIP6_CFsubhr.json',
 'CMIP6_E1hr.json',
 'CMIP6_E1hrClimMon.json',
 'CMIP6_E3hr.json',
 'CMIP6_E3hrPt.json',
 'CMIP6_E6hrZ.json',
 'CMIP6_Eday.json',
 'CMIP6_EdayZ.json',
 'CMIP6_Efx.json',
 'CMIP6_Emon.json',
 'CMIP6_EmonZ.json',
 'CMIP6_Esubhr.json',
 'CMIP6_Eyr.json',
 'CMIP6_IfxAnt.json',
 'CMIP6_IfxGre.json',
 'CMIP6_ImonAnt.json',
 'CMIP6_ImonGre.json',
 'CMIP6_IyrAnt.json',
 'CMIP6_IyrGre.json',
 'CMIP6_LImon.json',
 'CMIP6_Lmon.json',
 'CMIP6_Oclim.json',
 'CMIP6_Oday.json',
 'CMIP6_Odec.json',
 'CMIP6_Ofx.json',
 'CMIP6_Omon.json',
 'CMIP6_Oyr.json',
 'CMIP6_SIday.json',
 'CMIP6_SImon.json',
 'CMIP6_day.json',
 'CMIP6_fx.json']

In [142]:
tas = dr.get_all_variable_entries_by_attributes(
    out_name="tas", frequency="3hrPt", how="all"
)  # .keys()
tas

{'CMIP6_3hr.json': {'tas': {'frequency': '3hrPt',
   'modeling_realm': 'atmos',
   'standard_name': 'air_temperature',
   'units': 'K',
   'cell_methods': 'area: mean time: point',
   'cell_measures': 'area: areacella',
   'long_name': 'Near-Surface Air Temperature',
   'comment': 'near-surface (usually, 2 meter) air temperature',
   'dimensions': 'longitude latitude time1 height2m',
   'out_name': 'tas',
   'type': 'real',
   'positive': '',
   'valid_min': '',
   'valid_max': '',
   'ok_min_mean_abs': '',
   'ok_max_mean_abs': ''}}}

In [129]:
eval("all([True, False])") is True

False

In [138]:
tas["CMIP6_6hrPlev.json"]

{'tas': {'frequency': '6hr',
  'modeling_realm': 'atmos',
  'standard_name': 'air_temperature',
  'units': 'K',
  'cell_methods': 'area: time: mean',
  'cell_measures': 'area: areacella',
  'long_name': 'Near-Surface Air Temperature',
  'comment': 'near-surface (usually, 2 meter) air temperature',
  'dimensions': 'longitude latitude time height2m',
  'out_name': 'tas',
  'type': 'real',
  'positive': '',
  'valid_min': '',
  'valid_max': '',
  'ok_min_mean_abs': '',
  'ok_max_mean_abs': ''}}